## 10. Ограничения (PRIMARY KEY, FOREIGN KEY...)

### 🧱 Ограничения (PRIMARY KEY, FOREIGN KEY...)

**Постановка проблемы:**  
Чтобы база данных была надёжной, нужно контролировать, какие данные могут в неё попадать. Например, нельзя допустить повторяющийся ID, или чтобы книга ссылалась на несуществующий жанр. Для этого используются ограничения (constraints).

**Краткая теория:**  
Ограничения задаются при создании таблицы и проверяются автоматически при добавлении или изменении данных.

**Основные типы ограничений:**
- `PRIMARY KEY` — уникальный идентификатор строки (обычно `id`)
- `FOREIGN KEY` — связь с другой таблицей
- `NOT NULL` — запрет на пустые значения
- `UNIQUE` — значение должно быть уникальным
- `CHECK` — логическое условие (например, `year > 1900`)

**Пример:**

In [ ]:
# Пример ограничений в SQLite
import sqlite3

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Таблица авторов с UNIQUE и CHECK
cursor.execute('''
CREATE TABLE authors (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL UNIQUE,
    birth_year INTEGER CHECK (birth_year > 1800)
);
''')
cursor.executemany("INSERT INTO authors (name, birth_year) VALUES (?, ?)", [
    ("George Orwell", 1903),
    ("Aldous Huxley", 1894)
])

# Таблица книг с FOREIGN KEY
cursor.execute('''
CREATE TABLE books (
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    author_id INTEGER NOT NULL,
    FOREIGN KEY (author_id) REFERENCES authors(id)
);
''')
cursor.executemany("INSERT INTO books (title, author_id) VALUES (?, ?)", [
    ("1984", 1),
    ("Brave New World", 2)
])
conn.commit()

# Пробуем вставить с несуществующим author_id (ошибка)
try:
    cursor.execute("INSERT INTO books (title, author_id) VALUES (?, ?)", ("Fake Book", 99))
except sqlite3.IntegrityError as e:
    print("⛔ Ошибка ограничения:", e)

# Выводим таблицы
cursor.execute("SELECT * FROM authors")
print("👤 Авторы:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM books")
print("\n📚 Книги:")
for row in cursor.fetchall():
    print(row)